In [2]:
%reload_ext autoreload
%autoreload 2
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv
from gridworld_env_2d_state import GridworldEnv2DState
import torch
import random
import numpy as np
import math
from EXITrl.table_base import TableBase
from EXITrl.approx_v_base import ApproxVBase
from EXITrl.approx_policy_base import ApproxPolicyBase
from EXITrl.table_base import TableBase
from EXITrl.base import Base
import gym

In [12]:
class ActorCriticQ(ApproxVBase, ApproxPolicyBase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        hidden = 8
        features = torch.nn.Linear(self.num_state, hidden)
        model_q = torch.nn.Sequential(
            features,
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, self.num_action),
        ).to(self.device)
        model_policy = torch.nn.Sequential(
            features,
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, self.num_action),
        ).to(self.device)
        ApproxVBase.initialize(self, learning_rate_name="beta", model=model_q)
        ApproxPolicyBase.initialize(self, learning_rate_name="alpha", model=model_policy)
    
    def _loop(self, episode) -> int:
        done = False
        total_reward, reward = 0, 0
        state = self.env.reset()
        action, log_prob = self.policy(state)
        for i in range(1000):
            _state, reward, done, _ = self.env.step(action)
            _action, _log_prob = self.policy(_state)
            if done: 
                td_target = reward
            else: 
                td_target = reward + self.gamma * self.get_q(_state)[_action]
    
            # policy improvement
            # 1. Sutton's book
#             loss = (-log_prob) * self.gamma**i
            # 2. David's slide
            loss = (-log_prob) * self.get_q(state)[action]
            self.update_policy(loss)
            
            # value evaluation
#             td_error = td_target - self.get_q(state, action)
#             self.Q[state, action] += self.alpha * td_error
            current_q = self.get_q(state)[action]
            self.update_q(td_target, current_q)

            total_reward += reward
            state = _state
            action = _action
            log_prob = _log_prob
            if done: return total_reward

try: env.close()
except: pass
env = gym.make('CartPole-v1')
# env = GridworldEnv2DState()
# env = GridworldEnv()
s = ActorCriticQ(env, 
               num_episodes=500,
               policy="softmax_policy",
               alpha=0.007, 
               beta=0.1,
               gamma=.99)
s.train(True)
# s.convert_Q_to_V()


/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
episode: 0 reward: 9.0
episode: 1 reward: 15.0
episode: 2 reward: 11.0
episode: 3 reward: 14.0
episode: 4 reward: 13.0
episode: 5 reward: 14.0
episode: 6 reward: 8.0
episode: 7 reward: 9.0
episode: 8 reward: 12.0
episode: 9 reward: 12.0
episode: 10 reward: 10.0
episode: 11 reward: 9.0
episode: 12 reward: 12.0
episode: 13 reward: 9.0
episode: 14 reward: 9.0
episode: 15 reward: 10.0
episode: 16 reward: 9.0
episode: 17 reward: 9.0
episode: 18 reward: 9.0
episode: 19 reward: 11.0
episode: 20 reward: 9.0
episode: 21 reward: 12.0
episode: 22 reward: 12.0
episode: 23 reward: 10.0
episode: 24 reward: 9.0
episode: 25 reward: 10.0
episode: 26 reward: 10.0
episode: 27 reward: 10.0
episode: 28 reward: 9.0
episode: 29 reward: 9.0
episode: 30 reward: 12.0
episode: 31 reward: 11.0
episode: 32 reward: 9.0
episode: 33 reward: 9.0
episode: 34 reward: 10.0
episode: 35 reward: 10.0
episode: 36 reward: 9.0
ep

episode: 327 reward: 10.0
episode: 328 reward: 9.0
episode: 329 reward: 9.0
episode: 330 reward: 9.0
episode: 331 reward: 10.0
episode: 332 reward: 10.0
episode: 333 reward: 10.0
episode: 334 reward: 10.0
episode: 335 reward: 10.0
episode: 336 reward: 10.0
episode: 337 reward: 9.0
episode: 338 reward: 9.0
episode: 339 reward: 10.0
episode: 340 reward: 10.0
episode: 341 reward: 10.0
episode: 342 reward: 9.0
episode: 343 reward: 10.0
episode: 344 reward: 9.0
episode: 345 reward: 9.0
episode: 346 reward: 8.0
episode: 347 reward: 8.0
episode: 348 reward: 9.0
episode: 349 reward: 10.0
episode: 350 reward: 9.0
episode: 351 reward: 10.0
episode: 352 reward: 10.0
episode: 353 reward: 10.0
episode: 354 reward: 9.0
episode: 355 reward: 10.0
episode: 356 reward: 10.0
episode: 357 reward: 10.0
episode: 358 reward: 9.0
episode: 359 reward: 9.0
episode: 360 reward: 10.0
episode: 361 reward: 9.0
episode: 362 reward: 9.0
episode: 363 reward: 10.0
episode: 364 reward: 10.0
episode: 365 reward: 8.0
epis

### Continuous

In [4]:
import sklearn.pipeline
import sklearn.preprocessing
from sklearn.kernel_approximation import RBFSampler
# Feature Preprocessing: Normalize to zero mean and unit variance
# We use a few samples from the observation space to do this
env = gym.make('MountainCarContinuous-v0')
observation_examples = np.array([env.observation_space.sample() for x in range(10000)])
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(observation_examples)

# Used to converte a state to a featurizes represenation.
# We use RBF kernels with different variances to cover different parts of the space
featurizer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
        ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
        ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
        ("rbf4", RBFSampler(gamma=0.5, n_components=100))
        ])
featurizer.fit(scaler.transform(observation_examples))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


FeatureUnion(n_jobs=1,
       transformer_list=[('rbf1', RBFSampler(gamma=5.0, n_components=100, random_state=None)), ('rbf2', RBFSampler(gamma=2.0, n_components=100, random_state=None)), ('rbf3', RBFSampler(gamma=1.0, n_components=100, random_state=None)), ('rbf4', RBFSampler(gamma=0.5, n_components=100, random_state=None))],
       transformer_weights=None)

In [5]:
def featurize_state(state):
    """
    Returns the featurized representation for a state.
    """
    scaled = scaler.transform([state])
    featurized = featurizer.transform(scaled)
    return featurized[0]
state = env.reset()
featurize_state(state).shape

(400,)

In [6]:
from torch import nn
class GaussianPolicyModel(nn.Module):
    """TODO add to device()"""
    def __init__(self, num_input=2, num_output=1):
        super().__init__()
        hidden=8
        # 0. feature
        self.model_features = torch.nn.Sequential(
            torch.nn.Linear(num_input, hidden),
#             torch.nn.ReLU(),
#             torch.nn.Linear(hidden, hidden)
        )
        # 1. mu
        self.model_mu = torch.nn.Sequential(
#             torch.nn.Linear(hidden, hidden),
#             torch.nn.ReLU(),
            torch.nn.Linear(hidden, num_output)
        )
        # 2. sigma
        self.model_sigma = torch.nn.Sequential(
#             torch.nn.Linear(hidden, hidden),
#             torch.nn.ReLU(),
            torch.nn.Linear(hidden, num_output),
            torch.nn.Softplus() # make it positive, be careful threshold=20???
        )

    def forward(self, x):
        features = self.model_features(x)
        mu = self.model_mu(features)
        sigma = self.model_sigma(features)
        return mu, sigma
policy = GaussianPolicyModel(2)
# optimizer_sigma = torch.optim.Adam(policy.parameters(), lr=0.5)

In [15]:
class REINFORCE_continuous(ApproxVBase, ApproxPolicyBase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        hidden = 8
        model_q = torch.nn.Sequential(
            torch.nn.Linear(400, hidden),
#             torch.nn.ReLU(),
#             torch.nn.Linear(hidden, hidden),
#             torch.nn.ReLU(),
            torch.nn.Linear(hidden, 1),
        ).to(self.device)
        
        ApproxVBase.initialize(self, learning_rate_name="beta", model=model_q)
        ApproxPolicyBase.initialize(self, 
                                    learning_rate_name="alpha", 
                                    model=GaussianPolicyModel(400, self.num_action))
        
    def _loop(self, episode) -> int:
        done = False
        total_reward, reward = 0, 0
        state = self.env.reset()
        state = featurize_state(state)
        for i in range(1000):
            action, log_prob, entropy = self.policy(state)
#             print('\raction:',action,'***********' , end="")
            _state, reward, done, _ = self.env.step(action)
            _state = featurize_state(_state)
            self.env.render()
            
            if done: 
                td_target = reward
            else: 
                td_target = reward + self.gamma * self.get_v(_state)
            estimate_v = self.get_v(state)
            td_error = td_target - estimate_v
            
            self.update_v(td_target, self.get_v(state))
            loss = (-log_prob) * td_error # using the td error as our advantage estimate
            self.update_policy(loss)

            total_reward += reward
            state = _state
            if done: return total_reward

try: env.close()
except: pass
env = gym.make('MountainCarContinuous-v0')
s = REINFORCE_continuous(env, 
               num_episodes=1000,
               policy="gaussian_policy",
               alpha=0.0001, 
               beta=0.001,
               gamma=.99)
s.train(True)


/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
episode: 0 reward: 69.55597572101787
episode: 1 reward: 82.89091230924345
episode: 2 reward: 82.91766994364545


KeyboardInterrupt: 

In [10]:
env.close()